# Animations

Movies of several variables through the experiments

In [2]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean
import cosima_cookbook as cc
import dask.distributed as dsk
import glob
import matplotlib.gridspec as gs
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import scipy.stats as st
import xarray as xr
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from palettable.colorbrewer.diverging import RdBu_11_r as rdbu_cm
from joblib import Parallel, delayed

import warnings # ignore these warnings
warnings.filterwarnings("ignore", category = FutureWarning)
warnings.filterwarnings("ignore", category = UserWarning)
warnings.filterwarnings("ignore", category = RuntimeWarning)

figdir = '/home/561/jn8053/g_e14/figures-tmp/'

clnt = dsk.Client()
clnt

Client Scheduler: tcp://127.0.0.1:33793 Dashboard: /proxy/36925/status,Cluster Workers: 6 Cores: 24 Memory: 250.00 GiB


In [3]:
# Font size
plt.rcParams['font.size'] = 16
# Axes
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
# Saving parameters
plt.rcParams['savefig.dpi'] = 100
plt.rcParams['savefig.bbox'] = 'tight'

In [46]:
def figure_along_slope():
    
    fig, axs = plt.subplots(2, figsize = (15, 10))
    for ax in axs:
        ax.set_ylim(-80, -60);
        ax.set_xticks(np.arange(-250, 100, 50))
        ax.set_yticks(np.arange(-80, -55, 5))
    axs[0].set_title('UP')
    axs[1].set_title('DOWN')
    
    return fig, axs

In [17]:
session = cc.database.create_session()

In [22]:
mask_file = np.load('/g/data/e14/hd4873/runs/parcels/input/proj01/Antarctic_slope_contour_ht_1000m.npz')
shelf_mask = mask_file['contour_masked_above']
xt_ocean = mask_file['xt_ocean']
yt_ocean = mask_file['yt_ocean']
shelf_mask[shelf_mask != 0] = np.nan
shelf_mask[shelf_mask == 0] = 1
shelf_mask = xr.DataArray(shelf_mask, dims = ['yt_ocean', 'xt_ocean'], coords = {'yt_ocean':yt_ocean, 'xt_ocean':xt_ocean})

In [28]:
ht = cc.querying.getvar('01deg_jra55v13_ryf9091', 'ht', session, n = 1)

# SST

In [76]:
sst = {}
sst['ryf'] = cc.querying.getvar('01deg_jra55v13_ryf9091', 'surface_temp', session, frequency = '1 monthly', start_time = '2150-01-01', end_time = '2164-12-31').sel(time = slice('2150-01-01', None))
sst['eup'] = cc.querying.getvar('01deg_jra55v13_ryf9091_easterlies_up10', 'surface_temp', session, frequency = '1 monthly', start_time = '2150-01-01', end_time = '2164-12-31')
sst['edo'] = cc.querying.getvar('01deg_jra55v13_ryf9091_easterlies_down10', 'surface_temp', session, frequency = '1 monthly', start_time = '2150-01-01', end_time = '2164-12-31')

In [78]:
for k in ['ryf', 'eup', 'edo']:
    sst[k] = sst[k].sel(yt_ocean = slice(None, -50))*shelf_mask

In [93]:
def figures_parallel(i):
    fig, axs = figure_along_slope()
    
    axs[0].text(0.98, 0.08, str(sst['ryf'].isel(time = i)['time.year'].values)+'-'+str(sst['ryf'].isel(time = i)['time.month'].values), 
                horizontalalignment = 'right', transform = axs[0].transAxes);
    psi = axs[0].contourf(sst['ryf']['xt_ocean'], sst['ryf']['yt_ocean'], sst['eup'].isel(time = i)-sst['ryf'].isel(time = i), 
                          levels = np.arange(-.15, .16, .01), cmap = 'RdBu_r', extend = 'both')
    axs[0].contour(ht['xt_ocean'], ht['yt_ocean'], ht, levels = [1000, 3000], colors = 'k', linewidths = 0.5)
    
    axs[1].text(0.98, 0.08, str(sst['ryf'].isel(time = i)['time.year'].values)+'-'+str(sst['ryf'].isel(time = i)['time.month'].values), 
                horizontalalignment = 'right', transform = axs[1].transAxes);
    psi = axs[1].contourf(sst['ryf']['xt_ocean'], sst['ryf']['yt_ocean'], sst['edo'].isel(time = i)-sst['ryf'].isel(time = i), 
                          levels = np.arange(-.15, .16, .01), cmap = 'RdBu_r', extend = 'both')
    axs[1].contour(ht['xt_ocean'], ht['yt_ocean'], ht, levels = [1000, 3000], colors = 'k', linewidths = 0.5)
    
    fig.colorbar(psi, orientation = 'horizontal', shrink = .5).set_label('SSTa [$^{\circ}$C]')
    plt.savefig(figdir+'easterlies-collaborative/figs-to-animate/sst/sst-'+str(i).zfill(3)+'.png', facecolor = 'white')
    plt.close()

In [92]:
Parallel(n_jobs = -1)(delayed(figures_parallel)(n) for n in range(len(sst['ryf']['time'])));

# SSS

In [96]:
sss = {}
sss['ryf'] = cc.querying.getvar('01deg_jra55v13_ryf9091', 'surface_salt', session, frequency = '1 monthly', start_time = '2150-01-01', end_time = '2164-12-31').sel(time = slice('2150-01-01', None))
sss['eup'] = cc.querying.getvar('01deg_jra55v13_ryf9091_easterlies_up10', 'surface_salt', session, frequency = '1 monthly', start_time = '2150-01-01', end_time = '2164-12-31')
sss['edo'] = cc.querying.getvar('01deg_jra55v13_ryf9091_easterlies_down10', 'surface_salt', session, frequency = '1 monthly', start_time = '2150-01-01', end_time = '2164-12-31')

In [97]:
for k in ['ryf', 'eup', 'edo']:
    sss[k] = sss[k].sel(yt_ocean = slice(None, -50))*shelf_mask

In [99]:
def figures_parallel(i):
    fig, axs = figure_along_slope()
    
    axs[0].text(0.98, 0.08, str(sss['ryf'].isel(time = i)['time.year'].values)+'-'+str(sss['ryf'].isel(time = i)['time.month'].values), 
                horizontalalignment = 'right', transform = axs[0].transAxes);
    psi = axs[0].contourf(sss['ryf']['xt_ocean'], sss['ryf']['yt_ocean'], sss['eup'].isel(time = i)-sss['ryf'].isel(time = i), 
                          levels = np.arange(-.15, .16, .01), cmap = 'RdBu_r', extend = 'both')
    axs[0].contour(ht['xt_ocean'], ht['yt_ocean'], ht, levels = [1000, 3000], colors = 'k', linewidths = 0.5)
    
    axs[1].text(0.98, 0.08, str(sss['ryf'].isel(time = i)['time.year'].values)+'-'+str(sss['ryf'].isel(time = i)['time.month'].values), 
                horizontalalignment = 'right', transform = axs[1].transAxes);
    psi = axs[1].contourf(sss['ryf']['xt_ocean'], sss['ryf']['yt_ocean'], sss['edo'].isel(time = i)-sss['ryf'].isel(time = i), 
                          levels = np.arange(-.15, .16, .01), cmap = 'RdBu_r', extend = 'both')
    axs[1].contour(ht['xt_ocean'], ht['yt_ocean'], ht, levels = [1000, 3000], colors = 'k', linewidths = 0.5)
    
    fig.colorbar(psi, orientation = 'horizontal', shrink = .5).set_label('SSSa')
    plt.savefig(figdir+'easterlies-collaborative/figs-to-animate/sss/sss-'+str(i).zfill(3)+'.png', facecolor = 'white')
    plt.close()

In [100]:
Parallel(n_jobs = -1)(delayed(figures_parallel)(n) for n in range(len(sss['ryf']['time'])));